# <center><u> Project 3  Part 2 Core</center>
* Authored By: Eric N. Valdez
* Date: 2/11/2024

## <u>Part 2

Fore this phase of the project you will be helping Chcago law enforcement allocate resources for next 6 months.

You will compare the crime count forecast for <u>`2`</u> of the options below:
* Theft
* Battery
* Narcotics
* Assault
* Total Crime `(sum of all crime types).`

**`For each of your selected crimes, perform the following steps to develop a forecast.`**
* Transform the data to obtain a time series of the crime count <u>per month using .size().</u>
* Check for and address null values
* Decompose the Time Series to detect seaonality and decide whetehr to use a seasonal model or not
* Determine if nonseasonal and / or seasonal differencing is reqired
* Use the ACF and PACF plots of stationarity data to estimate initial orders
* Split the time series into training and test data `(Remember we want to predict 6 months)`
* Fit a manual ARIMA/SARIMA model based on the orders during your exploration.
    * Make forecast with your model.
    * Plot the forecasts versus the test data
    * Obtain metrics for evaluation
* Tune with pmdarima's auto_arima
    * Fit a model n training data with the best parameters from auto_arima
    * Obtain metrics for evaluation
    * Make forecasts with the auto_arima model
    * Plot the forecasts versus the test data 
* Select the final model and jsutify your choice. `(Which metrics or diagnostics support yourchoice?)`
* Make true future forecasts using the entire dataset `(for dates beyond the entire ts)`

**For each of your two selected crimes, answer the following questions in a markdown cell:**
* Calculate the predicted net change in the number `(raw counts)` of this type of crime from the beginning of the forecast to the end.
* Convert the raw delta into a percent change.  

**`Final Evaluation:`**

After completing both models and getting the forecasts for both crimes for 6 months into the future `(beyond the data),` answer the following quesitons in a <u>Markdown Cell:

### <u>Of the two crimes, which is forecasted to have the highest monthly count of crimes at the end of the forecast?
* `Answer:`

### <u>Of the two crimes, which is forecasted to have the highest net change by the end of the forecast?
* `Answer:`

### <u> Of the two crimes, which is forecasted to have the highest percent change by the end of the forecast?
* `Answer:`

**`Final Recomendations`**
* What recommendation would you make to the stakeholders?
* Include reporting_quality visual(s) to support your recommendation.